### Example 1 - Map of dissolved oxygen content in the North West Atlantic

This example shows how to read and manipulate Argo observations in parquet format. The data are stored across multiple files: we will load into memory only what we need by applying some filters, and we will create a map showing the dissolved oxygen measurements in the North West Atlantic.

This Example 1 shows how to use the full Argo dataset built from profile files (more details:). Example 2 shows how to use CrocoLake, which contains QCed data from different datasets (Argo, GLODAP, Spray Gliders as of today).


##### Note on parquet files
There are several ways to load parquet files in a dataframe in Python, and this notebook shows two of them:

1. pandas + pyarrow: this approach uses the pyarrow package to load the data into a pandas dataframe;
2. dask (+ pandas + pyarrow): this approach uses the dask package to load the data into a dask dataframe; it uses pandas and pyarrow under the hood, and a dask dataframe is (almost) identical to a pandas dataframe.

We will first use pyarrow to load the dataset, so that we can provide a target schema (containing the Argo variable names) and load the data consistently across floats, independently of what variables each float actually has. We will finally convert the dataset to a pandas dataframe for manipulation and plots.

Other ways to read parquet files are by using pandas (make sure you have pyarrow, fastparquet or some other suitable engine installed), and Dask. Generally speaking, you'll want to use Dask only if you need a large amount of data at the same time so that you can benefit from its parallelization. You should avoid Dask whenever the data fits in your RAM.

When reading parquet files with pyarrow (or pandas), you can either specificy the file name(s), or the directory containing all the parquet files. In latter case if you apply any filter, pandas and pyarrow will sort through all the files in the folder, reading into memory only the subsets that satisfy your filter.

#### Getting started

If you haven't already, install the required packages by running `pip install .` at the root of the repository.

We also need the dataset! In this example we use the Argo BGC dataset: you can uncomment and run the cell below, or copy-paste the command (without the leading `!`) in your command line. If you are interested in physical quantities only, you can replace 'BGC' with 'PHY' here and throughout the notebook.

In [ ]:
# !download_db -d Argo -t BGC --noqc

We then import the necessary modules and set up the path to the dataset (update the `parquet_dir` variable below if you have specified a different location in the previous cell or have moved the dataset).

In [ ]:
import datetime
import glob
from pprint import pprint

import pandas as pd
import pyarrow.parquet as pq

# Path to Argo BGC 'ALL'
parquet_dir = './CrocoLake/1010_BGC_ARGO-CLOUD-DEV/'
# Setting up parquet schema
BGC_schema = pq.read_schema(parquet_dir+"_common_metadata")

# Boundaries
lat0 = 15 
lat1 = 60
lon0 = -85
lon1 = -30

### pyarrow + pandas approach

We now create a `ParquetDataset` object that will allow us to read the data. Specifying a schema (that we read into `BGC_schema` in the previous cell) will make later operations faster.

We can get a peak at what variables are available in the dataset looking at its `schema` attribute. Note that we have not load any data into memory yet (except for the schema).

In [ ]:
dataset = pq.ParquetDataset(
    parquet_dir, 
    schema=BGC_schema
)
schema = dataset.schema
pprint(sorted(schema.names))

We now want to filter the dataset to load in memory only the data from the NWA (we set the values for `lat0`,`lat1`,`lon0`,`lon1` in the first cell) and recorded in the last 3 months.

The geographical coordinates are stored in the variables 'LATITUDE'and 'LONGITUDE'. We then generate the filter, with its syntax being: `[[(column, op, val), …],…]` where `column` is the variable name, and `val` is the value to for the operator `op`, which accepts `[==, =, >, >=, <, <=, !=, in, not in]`,

In [ ]:
reference_time = datetime.datetime.now() - datetime.timedelta(days=180)

filter_coords_time = [
    ("JULD",">",reference_time),
    ("LATITUDE",">",lat0), ("LATITUDE","<",lat1),
    ("LONGITUDE",">",lon0), ("LONGITUDE","<",lon1)
]

To get a pandas dataframe, we re-generate a `ParquetDataset` object adding the filters to it, and then we read it into a pandas dataframe with the `read().to_pandas()` methods of the dataset.

*NB:* The following operation still fetches a large amount of data (~2 GB), so you can reduce the number of days in filter above if you cannot use this much memory.

In [ ]:
%%time
ds = pq.ParquetDataset(parquet_dir, schema=BGC_schema, filters=filter_coords_time)
df = ds.read().to_pandas()
df

You can explore the dataframe just by calling it (`df`) as we did above. If you want a list of the variables that are stored, you can use `sorted(df.columns.to_list())`. The following cell shows that we loaded roughly 2 GB of data.

In [ ]:
memory_usage = df.memory_usage(deep=True).sum()/(1024**2)
print(f"DataFrame size: {memory_usage:.2f} MB")

If we now that we only need a subset of variables, we can specify them in the dataset's `read()` method. 

For example, let's say that we are interested in the adjusted measurements of the dissolved oxygen recorded in the past 6 months in the NWA. To exclude non valid and missing observations, we filter the dissolved oxygen (DOXY_ADJUSTED) to be within a (large) range of valid values. Besides the oxygen, we also want to keep the spatial and time coordinates.

In [ ]:
%%time
cols = ["LATITUDE","LONGITUDE","PRES_ADJUSTED","JULD","DOXY_ADJUSTED"]
filter_coords_time_doxy = [
    ("JULD",">",reference_time),
    ("LATITUDE",">",lat0), ("LATITUDE","<",lat1),
    ("LONGITUDE",">",lon0), ("LONGITUDE","<",lon1),
    ("DOXY_ADJUSTED",">=",-1e30),("DOXY_ADJUSTED","<=",+1e30)
]
ds = pq.ParquetDataset(parquet_dir, schema=BGC_schema, filters=filter_coords_time_doxy)
df = ds.read(columns=cols).to_pandas()
df

This operation was much faster (3-4x faster on my machine) and loaded a much smaller dataframe (~4.7 MB):

In [ ]:
memory_usage = df.memory_usage(deep=True).sum()/(1024**2)
print(f"DataFrame size: {memory_usage:.2f} MB")

#### Map

We can now produce a scatter plot as we'd normally do with pandas. For example, here is the average dissolved oxygen at each location in the dataframe that we loaded:

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import colormaps

ref_var = "DOXY_ADJUSTED"
# Group by 'LATITUDE' and 'LONGITUDE'
grouped = df.groupby(['LATITUDE', 'LONGITUDE']).agg({
    ref_var: 'mean'  # Take the mean intensity at that coordinate
}).reset_index()

# Plotting using Cartopy
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

# Scatter plot
cbar_min = grouped[ref_var].quantile(q=0.1)
cbar_max = grouped[ref_var].quantile(q=0.9)
plt.scatter(
    grouped['LONGITUDE'],
    grouped['LATITUDE'],
    c=grouped[ref_var],
    vmin=cbar_min,
    vmax=cbar_max,
    cmap='cividis',
    transform=ccrs.PlateCarree()
)
plt.colorbar(label='Average ' + ref_var)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('North-West Atlantic average ' + ref_var)
plt.grid(True)
plt.xlim([lon0, lon1])
plt.ylim([lat0, lat1])
plt.show()

### dask approach

dask can be faster at reading parquet databases thanks to its parallel and lazy evaluation of operations. 

When using dask, very few things change compared to using pandas and pyarrow: for example, dask dataframes are almost identical to pandas dataframes and indeed for our needs we will use the same syntax! The only difference is that dask does not evaluate the instructions immediately: it creates so-called delayed objects, through which it builds internally a graph of instructions optimized for the sequence of operations called. To trigger the actual computation we then need to call the `compute()` method of the dask dataframe. This allows dask to handle larger-than-memory datasets, by reading into memory only the portions needed to perform the optimized set of instructions.

We start by importing a few extra modules.

In [ ]:
import dask
import dask.dataframe as dd

We can use the same filters defined earlier, and we use `read_parquet()` to filter the database and prescribe what we will need.
Note that:
* unlike the pyarrow-pandas approach, here we provide both filters and columns in the same method;
* we provide the schema `BGC_schema` that we read during the set up of the excercise
* dask uses pyarrow under the hood (`engine` variable, other options are available)

In [ ]:
%%time
cols = ["LATITUDE","LONGITUDE","PRES_ADJUSTED","JULD","DOXY_ADJUSTED"]
filter_coords_time_doxy = [
    ("JULD",">",reference_time),
    ("LATITUDE",">",lat0), ("LATITUDE","<",lat1),
    ("LONGITUDE",">",lon0), ("LONGITUDE","<",lon1),
    ("DOXY_ADJUSTED",">=",-1e30),("DOXY_ADJUSTED","<=",+1e30)
]

ddf = dd.read_parquet(
    parquet_dir,
    engine="pyarrow",
    schema=BGC_schema,
    filters=filter_coords_time_doxy,
    columns= cols
)

It appears dask is much faster! Yet, it actually creates a delayed object, i.e. `ddf` contains the instructions that dask will later use to load the dataframe into memory. This allows us to use `ddf` to schedule all the operations that we'd normally perform on a dataframe. Eventually we will call `ddf.compute()` to actually evaluate the instructions.

If we look at `ddf`, the dataframe will in fact appear empty:

In [ ]:
ddf

We can explore its first entries with `head()`, which loads into memory only the first 5 entries:

In [ ]:
ddf.head()

To load the whole dataframe, we call `compute()` (this returns a pandas dataframe).

In [ ]:
%%time
ddf_loaded = ddf.compute()
ddf_loaded

It took just a few hundreds milliseconds! So yes, dask can be much faster than just using pyarrow and pandas.

If we look into `ddf_loaded` now, it will show the populated pandas dataframe.

(Note: if you compare `ddf_loaded` with `df` loaded with pyarrow, you'll see that the rows are not in the same order, yet under the dataframe you can see that the total number of rows is the same. Also the index seems smaller, but dask holds the dataframe in multiple partitions, and the index is reset at every partition.)

dask's main advantage is not just in loading the data faster, but in performing operations on larger-than-memory data. For example, it can compute the mean value of DOXY_ADJUSTED *in the whole Argo BGC dataset* without loading the whole data in memory (and pretty quickly, too):

In [ ]:
# note that we are not passing any filter conditions or selection of columns to load
ddf_all = dd.read_parquet(
    parquet_dir,
    engine="pyarrow",
    schema=BGC_schema,
)
doxy_mean = ddf_all['DOXY_ADJUSTED'].mean() # doxy_mean is a delayed object: it knows what operations to perform to compute the mean, 
                                            # but doesn't perform them until we call compute() in the next cell

In [ ]:
%%time
doxy_mean.compute()

#### Map

When producing the map with our dask approach, we would creat the `grouped` dataframe from the delayed dataframe `ddf`, and `compute()` it as late as possible.

In this example, the compute that we are triggering includes four operations:
* reading the filtered dataset;
* grouping the dataframe by geographical coordinates;
* averaging by pressure and time;
* resetting the index;

Dask internally builds a graph of all of these operations, so that it knows what subset of data is needed and it optimizes the number and sequence of instructions before executing them.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import colormaps

ref_var = "DOXY_ADJUSTED"
# Group by 'LATITUDE' and 'LONGITUDE', and aggregate
grouped = ddf.groupby(['LATITUDE', 'LONGITUDE']).agg({
    ref_var: 'mean'  # Take the mean intensity at that coordinate
}).reset_index()

grouped = grouped.compute() # no need to use compute() if you already did (e.g. if you are using ddf_loaded)

# Plotting using Cartopy
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

# Scatter plot
cbar_min = grouped[ref_var].quantile(q=0.1)
cbar_max = grouped[ref_var].quantile(q=0.9)
plt.scatter(
    grouped['LONGITUDE'],
    grouped['LATITUDE'],
    c=grouped[ref_var],
    vmin=cbar_min,
    vmax=cbar_max,
    cmap='cividis',
    transform=ccrs.PlateCarree()
)
plt.colorbar(label='Average ' + ref_var)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('North-West Atlantic average ' + ref_var)
plt.grid(True)
plt.xlim([lon0,lon1])
plt.ylim([lat0,lat1])
plt.show()

### Suggested exercises

Try and access some other data, for example:
* filtering by different time periods;
* mapping a different parameter;
* restraining the quality of the data further by imposing DOXY_ADJUSTED_QC equals to 1;
* performing reads/manipulations that you would need to perform your tasks.

If you encounter any issues, please [reach out](enrico.milanese@whoi.edu)!